In [23]:
import pandas as pd
import numpy as np

# Read the data from CSV
df = pd.read_csv('bicycle_network.csv')

# Determine the number of nodes
num_nodes = df[['node1', 'node2']].max().max() + 1

# Initialize the matrices
distance_matrix = np.full((num_nodes, num_nodes), np.inf)
scenic_beauty = np.zeros((num_nodes, num_nodes))
roughness = np.zeros((num_nodes, num_nodes))
safety = np.zeros((num_nodes, num_nodes))
slope = np.zeros((num_nodes, num_nodes))

# Populate the matrices with the data from the CSV
for _, row in df.iterrows():
    i, j = int(row['node1']), int(row['node2'])
    distance_matrix[i, j] = row['distance']
    scenic_beauty[i, j] = row['scenic_beauty']
    roughness[i, j] = row['roughness']
    safety[i, j] = row['safety']
    slope[i, j] = row['slope']

In [24]:
from scipy.optimize import linprog
import numpy as np
import networkx as nx

# Create the graph
G = nx.Graph()

for _, row in df.iterrows():
    G.add_edge(int(row['node1']), int(row['node2']), weight=row['distance'])

# Define the comfort criteria weights (adjust as needed)
weights = {
    'scenic_beauty': 1.0,
    'roughness': 1.0,
    'safety': 1.0,
    'slope': 1.0
}

# Aggregate the weights
def comfort_score(i, j):
    return (weights['scenic_beauty'] * scenic_beauty[i, j] +
            weights['roughness'] * roughness[i, j] +
            weights['safety'] * safety[i, j] +
            weights['slope'] * slope[i, j])

# Objective function: minimize total distance and maximize comfort
def objective(route):
    total_distance = sum(distance_matrix[route[i], route[i + 1]] for i in range(len(route) - 1))
    total_comfort = sum(comfort_score(route[i], route[i + 1]) for i in range(len(route) - 1))
    return total_distance - total_comfort  # maximize comfort by subtracting it

# Constraints
constraints = []

In [31]:
from deap import base, creator, tools, algorithms
import random

nodes_to_visit = [0, 2, 4, 6, 8, 10, 12, 14]  # Specify the nodes you want to visit

# Define the fitness function
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, nodes_to_visit, len(nodes_to_visit))
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    return objective(individual),

# Custom crossover function for handling subsets
def custom_crossover(ind1, ind2):
    size = min(len(ind1), len(ind2))
    cxpoint1 = random.randint(0, size - 1)
    cxpoint2 = random.randint(0, size - 1)
    
    if cxpoint2 < cxpoint1:
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1
    
    temp1 = ind1[cxpoint1:cxpoint2 + 1] + [item for item in ind2 if item not in ind1[cxpoint1:cxpoint2 + 1]]
    temp2 = ind2[cxpoint1:cxpoint2 + 1] + [item for item in ind1 if item not in ind2[cxpoint1:cxpoint2 + 1]]
    
    ind1[:], ind2[:] = temp1, temp2
    
    return ind1, ind2

toolbox.register("mate", custom_crossover)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Register functions to create individuals and the population
def init_individual(nodes_to_visit):
    return random.sample(nodes_to_visit, len(nodes_to_visit))

# Define a function to create a population based on the nodes to visit
def create_population(n, nodes_to_visit):
    population = []
    for _ in range(n):
        individual = toolbox.individual()
        individual[:] = init_individual(nodes_to_visit)
        population.append(individual)
    return population

# Example usage
population_size = 100

population = create_population(population_size, nodes_to_visit)
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=500, verbose=False)

# Get the best individual
best_individual = tools.selBest(population, 1)[0]
print("Best route:", best_individual)

Best route: [4, 10, 12, 8, 2, 0, 14, 6]
